In [1]:
import sys
import numpy as np
import pybamm
import matplotlib.pyplot as plt
sys.path.append('..')
from Parameter.Javid.parameters_Li_half import get_parameter_values

In [2]:
# set DFN half-cell model
model_half_cell = pybamm.lithium_ion.DFN({"working electrode": "positive"})

In [ ]:
# print variables
model_half_cell.default_var_pts

In [ ]:
# set number of points for spatial discretisation
var_pts = {
    "x_n": 3,
    "x_s": 30,
    "x_p": 30,
    "r_n": 3,
    "r_p": 100
}


In [ ]:
# set parameter values
parameter_values=pybamm.ParameterValues(get_parameter_values())

In [ ]:
# define GITT experiment
# GITT: short pulse followed by long rest period
pulse_duration = "10 minutes"  # duration of current pulse
rest_duration = "2 hours"      # relaxation time
current_rate = "0.05C"          # current during pulse
voltage_min = 2.5              # V - discharge cutoff
voltage_max = 4.2              # V - charge cutoff

# Create GITT discharge experiment
# Tuple groups discharge+rest as ONE cycle
experiment = pybamm.Experiment(
    [
        (
            f"Discharge at {current_rate} for {pulse_duration}",
            f"Rest for {rest_duration}"
        )
    ] * 50,  # repeat 50 pulse-rest cycles
    termination=f"{voltage_min} V"
)

In [ ]:
# set solver
solver = pybamm.CasadiSolver(mode="safe",max_steps=1000) # change to IDAKLUSolver for faster solving

In [ ]:
# run simulation 
sim = pybamm.Simulation(model_half_cell,experiment=experiment, parameter_values=parameter_values, solver=solver,var_pts=var_pts)
sol = sim.solve()

In [ ]:
# Plot GITT results
# Full voltage profile
sol.plot(["Voltage [V]", "Current [A]"])
plt.show()

# Voltage vs Capacity
fig, ax = plt.subplots(figsize=(10, 6))
time = sol["Time [h]"].entries
voltage = sol["Voltage [V]"].entries
capacity = sol["Discharge capacity [A.h]"].entries

ax.plot(capacity, voltage, 'b-', linewidth=1)
ax.set_xlabel('Capacity [Ah]')
ax.set_ylabel('Voltage [V]')
ax.set_title('GITT: Voltage vs Capacity')
ax.grid(True)
plt.show()

# Plot individual GITT cycles (cycles 1, 10, 20, 30, 40, 50)
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

cycle_indices = [0, 9, 19, 29, 39, 49]  # cycles 1, 10, 20, 30, 40, 50
for idx, i in enumerate(cycle_indices):
    cycle = sol.cycles[i]
    t = cycle["Time [h]"].entries
    v = cycle["Voltage [V]"].entries
    axes[idx].plot(t, v, 'b-', linewidth=2)
    axes[idx].set_xlabel('Time [h]')
    axes[idx].set_ylabel('Voltage [V]')
    axes[idx].set_title(f'Cycle {i+1}')
    axes[idx].grid(True)

plt.tight_layout()
plt.show()